In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pandasgui import show

- For heat correlations, Phik-matrix use recommended by Tuomas
- pd.melt
- to rotate the x-axis labels: plt.xticks(rotation=45); (the ; at the end important)
- Weather data
- Steel production, how much does the weather effect it?
- Multiline for visualization
    - highlights
- Research papers, compare different times of the year, 
- Compare good casts and bad casts
- Min() and Max() of the stopper

- Stopper position rapidly changing
- The flow throughput value
- This is when clogging happen

- When does clogging happen and how can we predict it?

https://www.veed.io

In [2]:
df = pd.read_csv('../data/sulatto_fulldata2025_version2.csv', delimiter=";", index_col=None)

C:\Users\kalle\AppData\Local\Temp\ipykernel_28464\1320071282.py:1: DtypeWarning: Columns (42) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/sulatto_fulldata2025_version2.csv', delimiter=";", index_col=None)


In [3]:
# load the weather data that matches partially with our steel data (originally FMI)
df_weather = pd.read_csv("../tornio_weather.csv")

In [4]:
df['End time'] = df['End time'].str[:-3]

In [5]:
df

,Material Type,End time,Route,Weight,Length,Thickness,Width,Grade,Quality Status,Seg No,...,CCM.MD. Level std_dev east Value,CCM.MD. Mold width Value,CCM.MIX. Concentration factor Value,CCM.STR. Speed - Mold avg Value,CCM.STR. Throughput Value,CCM.SLAB. Slab counter heat Value,CCM.SLAB. Slab counter sequence Value,HeatNo_in_Cast,SlabNo_in_Heat,CastNo
0,slab,2023-02-06 01:05,Route 1,25783,"10,63","202,000",1539,720-1,ERROR,1,...,"12,50",1539.0,"0,000","0,50","1,18",1.0,1.0,2821,1,100.0
1,slab,2023-02-06 01:05,Route 1,25783,"10,63","202,000",1539,720-1,ERROR,2,...,"12,50",1539.0,"0,000","0,54","1,28",1.0,1.0,2821,1,100.0
2,slab,2023-02-06 01:05,Route 1,25783,"10,63","202,000",1539,720-1,ERROR,3,...,"12,50",1539.0,"0,000","0,56","1,33",1.0,1.0,2821,1,100.0
3,slab,2023-02-06 01:05,Route 1,25783,"10,63","202,000",1539,720-1,ERROR,4,...,"12,50",1539.0,"0,000","0,57","1,37",1.0,1.0,2821,1,100.0
4,slab,2023-02-06 01:05,Route 1,25783,"10,63","202,000",1539,720-1,ERROR,5,...,"12,50",1539.0,"0,000","0,59","1,41",1.0,1.0,2821,1,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677810,slab,2023-04-30 16:03,Route 1,13145,"6,51","202,000",1284,720-1,ERROR,62,...,"5,50",1284.0,"0,000","0,30","0,64",8.0,53.0,1050,8,NaN
677811,slab,2023-04-30 16:03,Route 1,13145,"6,51","202,000",1284,720-1,ERROR,63,...,"5,50",1284.0,"0,000","0,27","0,56",8.0,53.0,1050,8,NaN
677812,slab,2023-04-30 16:03,Route 1,13145,"6,51","202,000",1284,720-1,ERROR,64,...,"5,50",1284.0,"0,000","0,26","0,51",8.0,53.0,1050,8,NaN
677813,slab,2023-04-30 16:03,Route 1,13145,"6,51","202,000",1284,720-1,ERROR,65,...,"5,50",1284.0,"0,000","0,24","0,45",8.0,53.0,1050,8,NaN


In [6]:
df['Route'].value_counts()

Route
Route 1    677815
Name: count, dtype: int64

In [7]:
df['CastNo'].nunique()

149

In [8]:
df['HeatNo_in_Cast'].nunique()

1011

In [9]:
df['CCM.SLAB. Slab counter heat Value'].value_counts()

CCM.SLAB. Slab counter heat Value
2.0    111623
3.0    111180
4.0    110786
1.0    109509
5.0    108686
6.0     88034
7.0     28377
8.0      5120
9.0       949
Name: count, dtype: int64

In [10]:
first_fifty = df[df['CastNo'].between(100, 149)]
first_fifty['CastNo'].nunique()

50

In [11]:
second_fifty = df[df['CastNo'].between(150, 199)]
second_fifty['CastNo'].nunique()

50

In [12]:
third_fifty = df[df['CastNo'].between(200, 249)]
third_fifty['CastNo'].nunique()

49

In [13]:
# we are going to use multi-line charts + df.melt() to visualize the process better
# see "ricebeefcoffee_revisited.ipynb" from lecture 9 examples on how to work
# with multiline + df.melt()

# list all columns you want to see in the multiline chart
columns_all = [
       'Length', 'Thickness', 'Width', 'Seg No',
       'CCM.TD. Heat counter Value',
       'CCM.TD. Temperature liquidus Value',
       'CCM.TD. Temperature liquidus Dynaphase Value',
       'CCM.TD. Steel weight in tundish Value',
       'CCM.TD. Tundish inflow Value', 'CCM.TD. Stopper Last Position Value',
       'CCM.TD. Stopper position max Value',
       'CCM.TD. Stopper position min Value',
       'CCM.TD. Stopper pos chg max segment Value',
       'CCM.TD. Stopper position chg 10 s Value',
       'CCM.TD. Stopper position slab chg Value', 'CCM.TD. Superheat Value',
       'CCM.TD. Temperature tundish Value', 
       'CCM.SEN. Immersion depth Value', 'CCM.MD. Level average Value',
       'CCM.MD. Level change (max-min) Value', 'CCM.MD. Level std_dev Value',
       'CCM.MD. Level std_dev west Value', 'CCM.MD. Level std_dev east Value',
       'CCM.MD. Mold width Value', 'CCM.MIX. Concentration factor Value',
       'CCM.STR. Speed - Mold avg Value', 'CCM.STR. Throughput Value',
       'CCM.SLAB. Slab counter heat Value',
       'CCM.SLAB. Slab counter sequence Value']

# here we make more specific set of columns for easier visualization

# IF WE TRY TO VISUALIZE ALL VARIABLES AT ONCE, we can see only straight
# lines because the chart has to zoom out too much

# group of columns with very high values
columns_high = ['CCM.MD. Level average Value', 'Air pressure mean [hPa]']

# group of columns with average value ranges 
columns_middle = ['CCM.TD. Stopper Last Position Value',
       'CCM.TD. Stopper position max Value',
       'CCM.TD. Stopper position min Value',
       'CCM.TD. Steel weight in tundish Value',
       'Relative humidity mean [%]']

# group of columns with small value ranges
columns_low = ['CCM.TD. Tundish inflow Value',
                'CCM.MD. Level change (max-min) Value', 'CCM.MD. Level std_dev Value',
       'CCM.MD. Level std_dev west Value', 'CCM.MD. Level std_dev east Value', 'CCM.MIX. Concentration factor Value',
       'CCM.STR. Speed - Mold avg Value', 'CCM.STR. Throughput Value',]


# even less columns in this version with small value ranges
columns_low2 = ['CCM.TD. Tundish inflow Value',
                'CCM.MD. Level std_dev Value',
       'CCM.STR. Speed - Mold avg Value', 'CCM.STR. Throughput Value', 'Dew-point temperature mean [°C]']

In [14]:
df['CastNo'].value_counts()

CastNo
122.0    7400
178.0    7308
152.0    7270
142.0    7218
134.0    7122
         ... 
138.0     584
100.0     552
101.0     540
169.0     342
181.0      60
Name: count, Length: 149, dtype: int64

#### Create melt versions of some of the columns defined above

In [15]:
df.iloc[4376]

Material Type                                               slab
End time                                        2023-02-07 21:59
Route                                                    Route 1
Weight                                                     25793
Length                                                     10,63
Thickness                                                202,000
Width                                                       1543
Grade                                                      720-1
Quality Status                                                OK
Seg No                                                        88
Seg from                                                     8,7
Seg to                                                       8,8
CCM.TD. Heat counter Value                                   6.0
CCM.TD. Temperature liquidus Value                        1455.0
CCM.TD. Temperature liquidus Dynaphase Value              1460.0
CCM.TD. TD internals type

In [16]:
# for i in df['CastNo'].unique():

#     selected_id = i
    
#     if selected_id == 150.:
#         break
#     df_single = df[df['CastNo'] == selected_id]
    
#     df_single['ProcessNo'] = df_single['HeatNo_in_Cast'].astype(str) + \
#                          df_single['SlabNo_in_Heat'].astype(str).str.zfill(2) + \
#                          df_single['Seg No'].astype(str).str.zfill(2) 


#     df_melt_high = pd.melt(df_single, id_vars=["ProcessNo"], value_vars=columns_high,
#                   var_name="Measurement", value_name="Value")
    
#     fig, axes = plt.subplots(figsize=(12, 8))

#     plot = sns.lineplot(data=df_melt_high, x="ProcessNo", y="Value", hue="Measurement", errorbar=None)
#     plot.invert_yaxis()
#     sns.move_legend(plot, "upper left", bbox_to_anchor=(1,1))

#     # reduce the number of x-labels
#     n = len(df_single) / 20
#     [l.set_visible(False) for (i, l) in enumerate(plot.xaxis.get_ticklabels()) if i % n != 0]

#     # NOTE! using ; in the following line will suppress the Jupyter Notebook 
#     # default printing behavior of the last line (no idea why this happens!)
#     plt.xticks(rotation=45);
#     # Adjust layout to prevent overlapping titles
#     plt.tight_layout()
#     plt.title(f'Cast: {i}')
    
#     filename = os.path.join('graphs/high_graphs', f'cast_{i}.png')

#     plt.savefig(filename)

#     plt.close()


    


In [17]:
df['CCM.TD. Temperature liquidus Dynaphase Value'].nunique()

10